# Lab 11: Integer Linear programming and Dinamic Programming 
In this lab, we will see Integer Linear programming and Dynamic programming (DP). 

For the Integer Linear Programming, you will proceed as in the previous lab, writing some models and solving the problem. For Dynamic Programming, we will ask you to solve the knapsack problem with DP. 

In [1]:

!pip install -q condacolab
import condacolab
condacolab.install()
!conda install pyscipopt

✨🍰✨ Everything looks OK!
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - done

# All requested packages already installed.



In [2]:
!pip install scikit-optimize
!pip install treed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pyscipopt
from pyscipopt import Model , quicksum
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pyscipopt import Model, Eventhdlr, SCIP_EVENTTYPE
import math
from treed import TreeD

class LPstatEventhdlr(Eventhdlr):
    """PySCIPOpt Event handler to collect data on LP events."""

    transvars = {}

    def collectNodeInfo(self, firstlp=True):
        objval = self.model.getSolObjVal(None)

        LPsol = {}
        if self.transvars == {}:
            self.transvars = self.model.getVars(transformed=True)
        for var in self.transvars:
            solval = self.model.getSolVal(None, var)
            LPsol[var.name] = self.model.getSolVal(None, var)

        # skip duplicate nodes
        # if self.nodelist and LPsol == self.nodelist[-1].get("LPsol"):
        #     return
        node = self.model.getCurrentNode()
        if node.getNumber() != 1:
            parentnode = node.getParent()
            parent = parentnode.getNumber()
        else:
            parent = 1
        depth = node.getDepth()
        age = self.model.getNNodes()
        condition = math.log10(self.model.getCondition())
        iters = self.model.lpiGetIterations()
        pb = self.model.getPrimalbound()
        if pb >= self.model.infinity():
            pb = None

        nodedict = {
            "number": node.getNumber(),
            "LPsol": LPsol,
            "objval": objval,
            "parent": parent,
            "age": age,
            "depth": depth,
            "first": firstlp,
            "condition": condition,
            "iterations": iters,
            # "variables": self.model.getNVars(),
            # "constraints": self.model.getNConss(),
            "rows": self.model.getNLPRows(),
            "primalbound": pb,
            "dualbound": self.model.getDualbound(),
            "time": self.model.getSolvingTime()
        }

        self.nodelist.append(nodedict)

    def eventexec(self, event):
        
        if event.getType() == SCIP_EVENTTYPE.FIRSTLPSOLVED:
            self.collectNodeInfo(firstlp=True)
        elif event.getType() == SCIP_EVENTTYPE.LPSOLVED:
            self.collectNodeInfo(firstlp=False)
        else:
            print("unexpected event:" + str(event))
        return {}

    def eventinit(self):
        self.model.catchEvent(SCIP_EVENTTYPE.LPEVENT, self)

def convertSolToDict(sol):
    ssol = str(sol)
    items = ssol[1:-1].split(",")
    res = {}
    for item in items:
        k = str(item.split(":")[0])
        v = float(item.split(":")[1])
        res[k] = v

def solve_model(model):
    model.setPresolve(pyscipopt.SCIP_PARAMSETTING.OFF)
    model.setHeuristics(pyscipopt.SCIP_PARAMSETTING.OFF)
    model.disablePropagation()
    #model.redirectOutput()
    nodelist = []
    eventhdlr = LPstatEventhdlr()
    eventhdlr.nodelist = nodelist
    model.includeEventhdlr(
        eventhdlr, "LPstat", "generate LP statistics after every LP event"
    )
    model.optimize()
    frontier_history = []
    for nd in nodelist:
        frontier_history.append((convertSolToDict(nd['LPsol']), nd['objval']))
    best_sol = model.getBestSol()
    best_val = model.getObjVal()
    return convertSolToDict(best_sol), best_val, frontier_history

#Ex. 0

This is an example to show how the Library works
---

Slack form:

minimize  $2x_1 + x_2 − 2x_3$

subject to 

> $0.7x_1 + 0.5x_2 +x_3 ≥ 1.8$

> $x_i ∈ [0,1]\ ∀ i$





In [4]:
model = Model("")
v = [model.addVar(vtype='i', name=str(i),lb=0,ub=1) for i in range(3)]
c = [0.7,0.5,1]
model.addCons(quicksum(c[i]*v[i] for i in range(3))>=1.8)
model.setObjective(2*v[0]+v[1]-2*v[2], "minimize")
best_sol, best_val, partial_frontier_history = solve_model(model)

#Ex. The N-queens Problem | ILP

The N-queens Problem | ILP
The N queens puzzle is the problem of placing N chess queens on an $N×N (𝑁≥4)$ chessboard so that no two queens threaten each other; thus, a solution requires that no two queens share the same row, column, or diagonal. Try different values of $𝑁$ and shows how the problem complexity increases.

```c
param n, integer, > 0, default 16;
/* size of the chess board */
var x{1..n, 1..n}, binary;
/* x[i,j] = 1 means that a queen is placed in square [i,j] */
s.t. a: sum{i in 1..n} sum{j in 1..n} x[i,j] == n;
/*total of n Queens. It's the objective function */
s.t. b{i in 1..n}: sum{j in 1..n} x[i,j] <= 1;
/* at most one queen can be placed in each row */
s.t. c{j in 1..n}: sum{i in 1..n} x[i,j] <= 1;
/* at most one queen can be placed in each column */
s.t. d{k in 2-n..n-2}: sum{i in 1..n, j in 1..n: i-j == k} x[i,j] <= 1;
/* at most one queen can be placed in each "\"-diagonal */
s.t. e{k in 3..n+n-1}: sum{i in 1..n, j in 1..n: i+j == k} x[i,j] <= 1;
/* at most one queen can be placed in each "/"-diagonal */
/* solve the problem */
```

In [7]:
n = 4
model = Model("")
v = [model.addVar(vtype='i',name=str(i), lb=0, ub=1) for i in range(3)]
c = [1,1,1]
model.addCons(quicksum(c[i]*v[i] for i in range(3)) == 1)
model.setObjective(quicksum(v[i][j] for i in range(n) for j in range(n)), "minimize")
best_sol, best_val, partial_frontier_history = solve_model(model)

AttributeError: ignored

# Traveling salesman problem | ILP
The goal of the TSP is to find the shortest Hamiltonian cycle (a cycle that visits each node only once) on a graph of N nodes. Solve the ILP problem and visualize a solution.

$$
minimize\\
\sum_{i=1}^{n}\sum_{j=1}^{n} x_{ij}c_{ij}\\
subject\ to \\
\sum_{1 \leq i \leq n\\ i \neq j} x_{ij} = 1
\sum_{1 \leq k \leq n\\ k \neq j} x_{jk} = 1
$$

for each sum in the constraint it's for each $j = 1,2,...,n$ 

#Ex. KNAPSACK PROBLEM / DP
Solve the Knapsack problem using DP.  You can use the Class from lab 5.

Run the solution multiple time, and change the total capacity and the number of objects and shows how the number of subproblems changes.


In [8]:
import numpy as np
from matplotlib import pyplot as plt

class Knapsack_0_1:
    
    def __init__(self):
        self._items = [
                        []   
        ]
        self._BAG_CAPACITY = 10
        self.history = []
        self.values = []

    def _get_value(self, solution):
        cur_cap = self._BAG_CAPACITY
        cur_val = 0
        for i, v in enumerate(solution):
            if v == 1:
                cur_val += self._items[i]['value']
                cur_cap -= self._items[i]['volume']
            if cur_cap < 0:
                return 0
        return -cur_val

    def __call__(self, solution):
        value = self._get_value(solution)
        self.history.append(solution)
        self.values.append(value)
        return value
    
    def trend(self):
        plt.figure()
        plt.plot(self.values)
        plt.show()
